In [1]:
import time
from cffi import FFI
from array2gif import write_gif
import numpy as np
import random

# options
# this program requires ~ 5000 cycles
cycles      = 5000 # cpu cycles
binary      = 'sierp0.bin' # program
outputfile  = 'sierp0.gif'
framebuffer = 0x200 # framebuffer offset
pc          = 0x600 # program counter on reset
debug       = 0 # show register state during execution

# prog = [
#  0xa2,0x00,0xa9,0x00,0x85,0x00,0xa9,0x02,
#  0x85,0x01,0x20,0x1f,0x06,0x81,0x00,0xe6,
#  0x00,0xf0,0x03,0x4c,0x0a,0x06,0xe6,0x01,
#  0xa4,0x01,0xc0,0x06,0xd0,0xec,0x60,0xa5,
#  0x00,0x29,0x1f,0x85,0x02,0xa5,0x00,0x4a,
#  0x4a,0x4a,0x4a,0x4a,0x85,0x03,0xa5,0x01,
#  0x38,0xe9,0x02,0x0a,0x0a,0x0a,0x05,0x03,
#  0x25,0x02,0xf0,0x03,0xa9,0x02,0x60,0xa9,
#  0x0d,0x60]

with open(binary, "rb") as f:
    prog = f.read()
    
# init 6502
def init(ffi, PC, SP):
    _6502 = ffi.dlopen("./6502.so");
    _6502.reset(PC,SP); _6502.show_debug=debug
    return _6502

In [2]:
def execute(prog):

    ffi = FFI()

    #C header stuff
    ffi.cdef("""
       typedef uint8_t u8; typedef uint16_t u16; typedef uint32_t u32; typedef uint64_t u64;
       void reset(u16,u8); u8 show_debug;
       void cpu_step(u32);
       extern u8 mem[0x10000];
       extern u8 show_debug;
    """)

    #initial program counter and stack pointer
    PC=pc; SP=0xff; FB=framebuffer

    C = init(ffi, PC, SP);
    n_frames = cycles//40

    for i in range(len(prog)): C.mem[PC+i] = prog[i]

    if outputfile:
        imgs = []
        for jj in range(n_frames):
            C.cpu_step(cycles // n_frames)
            _scr = np.zeros((2048,1), dtype='uint8')
            for i in range(512): _scr[i] = C.mem[i] # zero pg,stack
            for i in range(512): _scr[i+512] = C.mem[PC+i] # program
            for i in range(1024): _scr[i+1024] = 255 * C.mem[FB+i]  # 'screen'screen
            frame = _scr.reshape(1,64,32);
            zp = frame[:,0:32,:]; prg = frame[:,32:64,:]
            frame = np.zeros((1,32,64))
            frame[:,:,0:32] = zp; frame[:,:,32:64] = prg
            ff = frame.repeat(3, axis=0)
            imgs.append(ff);
        return imgs

    else:
        C.cpu_step(cycles) # just run for n cycles

        # todo, this is temporary
        _scr = np.zeros((1024,1), dtype='uint8')
        for i in range(1024): _scr[i] = C.mem[FB+i]
        
        return _scr.reshape(1,32,32)

In [3]:
frames = execute(prog)
write_gif(frames, outputfile, fps=25)

from IPython.display import HTML
HTML('<img src="sierp0.gif" style="width:500px;height:300px;">')

/home/krocki/.local/lib/python3.8/site-packages/cffi/cparser.py:163: UserWarning: Global variable 'show_debug' in cdef(): for consistency with C it should have a storage class specifier (usually 'extern')
  warnings.warn("Global variable '%s' in cdef(): for consistency "
